# **Token per second: 1.54 ; 1.59**

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git accelerate
!pip install huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 21.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from huggingface_hub import snapshot_download

token = "hf_token"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

checkpoint_path = snapshot_download(repo_id=model_id, use_auth_token=token)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=token)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        checkpoint_path,
        torch_dtype=torch.bfloat16
    )

# Specify memory limits
max_memory = {0: "15GB", "cpu": "13GB"}

model = load_checkpoint_and_dispatch(
    model,
    checkpoint_path,
    device_map="auto",
    max_memory=max_memory,
    offload_folder="offload",
    offload_state_dict=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

torch.cuda.empty_cache()
torch.cuda.memory.empty_cache()

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

USE_POLICY.md:   0%|          | 0.00/4.69k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?w/s]

  0%|          | 0/104 [00:00<?, ?w/s]

  0%|          | 0/100 [00:00<?, ?w/s]

  0%|          | 0/5 [00:00<?, ?w/s]

In [ ]:
def generate_in_chunks(prompt, total_tokens, chunk_size=60):
    full_output = prompt
    remaining_tokens = total_tokens

    while remaining_tokens > 0:
        chunk_tokens = min(chunk_size, remaining_tokens)
        output = pipe(full_output, max_new_tokens=chunk_tokens, do_sample=True, temperature=0.7)
        new_text = output[0]['generated_text'][len(full_output):]
        full_output += new_text
        remaining_tokens -= chunk_tokens

    return full_output

prompt = "Can you calculate the square root of 40 ?"
result = generate_in_chunks(prompt, total_tokens=210, chunk_size=70)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Can you calculate the square root of 40 ? The answer is not a whole number, so we have to use a calculator to find the square root of 40. The square root of 40 is 6.32455532. We can round this to 6.3.
We can multiply 6.3 by itself to find the square of 6.3.
6.3 × 6.3 = 39.69
So the square of 6.3 is 39.69. We can see that 6.3 is a close approximation of the square root of 40.
What is the square root of 40?
Answer: 6.32455532
The square of 6.3 is 39.69.
Answer: 39.69
Can you calculate the square root of 36? The answer is a whole number, so we can do this calculation by hand. We can find the square root of 36 by finding the number that we can multiply by itself to get 36.
36 ÷ 6 = 
